<a href="https://colab.research.google.com/github/denverkim/Uzbek/blob/main/23_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai chromadb langchain langchain-openai langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.8 MB/s eta 0:00:0

In [2]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import PromptTemplate

In [3]:
# 1. OpenAI LLM setup

from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.2, api_key=OPENAI_API_KEY)

In [4]:
# 2. FAQ document data (sample)
docs = [
    "Q: How do I submit an approval request? A: In the ERP system, go to the [Approval] menu and create a request form.",
    "Q: What is the procedure for requesting vacation? A: In the groupware [Attendance Management] menu, fill out a vacation request form and get approval from your supervisor.",
    "Q: Who should I contact for HR-related inquiries? A: You can contact the HR manager in the HR team by email."
]

In [5]:
# 3. Embedding & vector store creation
embeddings = OpenAIEmbeddings(model="text-embedding-3-small",
                              api_key=OPENAI_API_KEY)
vector_db = Chroma.from_texts(docs, embeddings)

In [6]:
# 4. Define user query
user_query = "I want to take a vacation, how can I apply?"

In [9]:
# 5. RAG-style search
related_doc = vector_db.similarity_search(user_query, k=1)[0].page_content
related_doc

'Q: What is the procedure for requesting vacation? A: In the groupware [Attendance Management] menu, fill out a vacation request form and get approval from your supervisor.'

In [8]:
# 6. Prompt creation
prompt_template = PromptTemplate.from_template("""
Refer to the following document and answer the user's question naturally.
Document: {context}
Question: {question}
""")
filled_prompt = prompt_template.format(context=related_doc, question=user_query)

# 7. Generate LLM response
response = llm.invoke(filled_prompt)

print("Retrieved Document:", related_doc)
print("Generated Answer:", response.content)


Retrieved Document: Q: What is the procedure for requesting vacation? A: In the groupware [Attendance Management] menu, fill out a vacation request form and get approval from your supervisor.
Generated Answer: To apply for a vacation, you need to go to the groupware under the [Attendance Management] menu. There, you should fill out a vacation request form and then get it approved by your supervisor.


In [10]:
filled_prompt

"\nRefer to the following document and answer the user's question naturally.\nDocument: Q: What is the procedure for requesting vacation? A: In the groupware [Attendance Management] menu, fill out a vacation request form and get approval from your supervisor.\nQuestion: I want to take a vacation, how can I apply?\n"